In [3]:
from bs4 import BeautifulSoup as soup
import os
import os.path
import glob
import time
import requests
import random
from tools import GetAgent, GetLastPage, GetPhoneNum, DownloadImg, CreatePages, GetCategory, GetImgUrl, GetNameAndLink, GetAddr

因為是要抓取台灣的店家清單，所以先從搜尋的首頁抓出總頁數，再把頁數建立成一個list

In [4]:
main_url = 'http://www.ipeen.com.tw/search/taiwan/000/1-100-0-0/'
result = requests.get(main_url)
contents = soup(result.text, 'lxml')
last_page = contents.find_all('a',{'data-label':'最尾頁'})
last_page_url = last_page[0].attrs['href']
page_count = last_page_url.split('=')[1]
url_list = []
for count in range(1, int(page_count)):
    url_list.append(last_page_url.split('=')[0]+str(count))
print(len(url_list))

11246


從頁數的list中，開始針對各分頁進行爬蟲，並把爬出來的結果寫入檔案中

In [ ]:
for this_page in url_list:
    page_cont = requests.get(this_page, GetAgent())
    cont = soup(page_cont.text, 'lxml')
    print(this_page)
    articles = cont.find_all('article')
    with open('ipeen_detail.csv', 'a+', encoding ='utf8') as record:
        this_page = []
        for article in articles:
            addr = GetAddr(article)
            img = GetImgUrl(article)
            name = img['name']
            link = img['link']
            print(img)
            phone = ''
            if 'phone' in img.keys():
                phone = img['phone']
            categories = GetCategory(article)
            cate1 = categories['category1']
            cate2 = categories['category2']
            all_info = name+','+addr+','+phone+','+cate1+','+cate2+'\n'
            if (len(all_info) == 0):
                this_page.append(all_info)
            elif all_info not in this_page:
                this_page.append(all_info)
            t = int(random.random() * 10)
            time.sleep(t)
        for p in this_page:
            record.write(p)
        record.flush()
    